In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score


def transform_data(dataset):
    train1 = dataset[['pid','Age','ABPm','Heartrate','SpO2','ABPs','Temp']].groupby('pid',as_index=False,sort=False).mean()
    train2 = dataset[['pid','ABPm','SpO2','Heartrate','ABPs','Temp','BaseExcess']].groupby('pid',as_index=False,sort=False).std()
    train3 = dataset[['pid','EtCO2','PTT','BUN','Lactate','Hgb','HCO3','BaseExcess','Fibrinogen','Phosphate','WBC','Creatinine','PaCO2','AST','FiO2','Platelets','SaO2','Glucose','Magnesium','Potassium','Calcium','Alkalinephos','Bilirubin_direct','Chloride','Hct','Bilirubin_total','TroponinI','pH']].drop('pid',1).isna().groupby(dataset.pid,as_index=False,sort=False).sum()
    train_features = pd.concat([train1,train2,train3],axis=1)
    train_features = train_features.drop(['pid'],axis=1)
    imputer = KNNImputer(n_neighbors=5,missing_values=np.nan)
    train_features = imputer.fit_transform(train_features)
    return train_features

In [3]:
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv',usecols=['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2','LABEL_Sepsis'])
test_features = pd.read_csv('test_features.csv')
pid = test_features['pid'].unique()
test_features = transform_data(test_features)
train_features = pd.DataFrame(transform_data(train_features))
train_features.shape

/var/folders/k1/hf4djm2s1klctv2qbryz5kb80000gn/T/ipykernel_40059/3108425677.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train3 = dataset[['pid','EtCO2','PTT','BUN','Lactate','Hgb','HCO3','BaseExcess','Fibrinogen','Phosphate','WBC','Creatinine','PaCO2','AST','FiO2','Platelets','SaO2','Glucose','Magnesium','Potassium','Calcium','Alkalinephos','Bilirubin_direct','Chloride','Hct','Bilirubin_total','TroponinI','pH']].drop('pid',1).isna().groupby(dataset.pid,as_index=False,sort=False).sum()
/var/folders/k1/hf4djm2s1klctv2qbryz5kb80000gn/T/ipykernel_40059/3108425677.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train3 = dataset[['pid','EtCO2','PTT','BUN','Lactate','Hgb','HCO3','BaseExcess','Fibrinogen','Phosphate','WBC','Creatinine','PaCO2','AST','FiO2','Platelets','SaO2','Glucose','Magnesium','Potassium

(18995, 39)

In [284]:
out_test = pd.DataFrame()
out_test['pid'] = pid
y_score = pd.DataFrame()

for i in train_labels.columns:
    sm = SMOTE()
    clf = RandomForestClassifier(n_jobs=-1)
    x,y = sm.fit_resample(train_features,train_labels[i])
    clf.fit(x,y)
    proba_test_features = np.take(clf.predict_proba(test_features),(1),axis=1)
    out_test[i] = proba_test_features
    
labels_3 = pd.read_csv('train_labels.csv',usecols=['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate'])
model = Ridge()
out_task3 = pd.DataFrame()
for i in labels_3.columns:
    x = train_features
    y = labels_3[i]
    model.fit(x,y)
    Y = model.predict(test_features)
    out_test[i] = Y
    
out_test.to_csv('result.zip',index=False,float_format='%.3f',compression='zip')